In [87]:
import numpy as np
from Chempy.parameter import ModelParameters
import importlib
import fileinput
import sys   

In [91]:
# Starting elements (copied from original parameter file)
elements_to_trace = ['Al', 'Ar', 'B', 'Be', 'C', 'Ca', 'Cl', 'Co', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'Ge', 'H', 'He', 'K', 'Li', 'Mg', 'Mn', 'N', 'Na', 'Ne', 'Ni', 'O', 'P', 'S', 'Sc', 'Si', 'Ti', 'V', 'Zn']
orig = "\telements_to_trace = "+str(elements_to_trace) # Original element string

## STILL NOT WORKING FOR SOME ELEMENTS!!!!??

# Calculate required Chempy elements
sol_dat = np.load("Chempy/input/stars/Proto-sun.npy")
el_names = []
for el in all_elements:
    if el in sol_dat.dtype.names:
        el_names.append(el)

for i in range(len(el_names)): # Iterate over removed element
    for line in fileinput.input("Chempy/parameter.py", inplace=True):
        if "\telements_to_trace" in line:
            if el_names[i] == 'Zn':
                newline = line.replace("'"+el_names[i]+"'","") # No comma
                print(newline,end='')
            else:
                newline = line.replace("'"+el_names[i]+"', ","")
                print(newline,end='')
        else:
            print(line,end='')
        # ALSO change it to only remove elements in a.elements_ AND the wildcard
    # DO WHATEVER YOU WANT
    del sys.modules['Chempy.parameter']
    from Chempy.parameter import ModelParameters
    a = ModelParameters()
    print(el_names[i])
    print(a.elements_to_trace)
    ##############
    # Now we are free to run new code
    
    ##############
    for line in fileinput.input("Chempy/parameter.py", inplace=True):
        if "\telements_to_trace" in line:
            print(orig)
        else:
            print(line,end='')

Al
['Ar', 'B', 'Be', 'C', 'Ca', 'Cl', 'Co', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'Ge', 'H', 'He', 'K', 'Li', 'Mg', 'Mn', 'N', 'Na', 'Ne', 'Ni', 'O', 'P', 'S', 'Sc', 'Si', 'Ti', 'V', 'Zn']
Ar
['Ar', 'B', 'Be', 'C', 'Ca', 'Cl', 'Co', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'Ge', 'H', 'He', 'K', 'Li', 'Mg', 'Mn', 'N', 'Na', 'Ne', 'Ni', 'O', 'P', 'S', 'Sc', 'Si', 'Ti', 'V', 'Zn']
C
['Al', 'Ar', 'B', 'Be', 'Ca', 'Cl', 'Co', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'Ge', 'H', 'He', 'K', 'Li', 'Mg', 'Mn', 'N', 'Na', 'Ne', 'Ni', 'O', 'P', 'S', 'Sc', 'Si', 'Ti', 'V', 'Zn']
Ca
['Al', 'Ar', 'B', 'Be', 'C', 'Cl', 'Co', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'Ge', 'H', 'He', 'K', 'Li', 'Mg', 'Mn', 'N', 'Na', 'Ne', 'Ni', 'O', 'P', 'S', 'Sc', 'Si', 'Ti', 'V', 'Zn']
Co
['Al', 'Ar', 'B', 'Be', 'C', 'Cl', 'Co', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'Ge', 'H', 'He', 'K', 'Li', 'Mg', 'Mn', 'N', 'Na', 'Ne', 'Ni', 'O', 'P', 'S', 'Sc', 'Si', 'Ti', 'V', 'Zn']
Cr
['Al', 'Ar', 'B', 'Be', 'C', 'Cl', 'Co', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'Ge', 'H', 'He', 'K', 'Li',